In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
testing_data = pd.read_csv('/content/drive/MyDrive/Intact data/new_test.csv')

In [ ]:
testing_data.head()

,Unnamed: 0,transcription
0,0,"PAST MEDICAL CONDITION:, None.,ALLERGIES:, N..."
1,1,"CURRENT HISTORY:, A 94-year-old female from t..."
2,2,"TITLE OF PROCEDURE: ,Coronary artery bypass g..."
3,3,"PROCEDURE: , Bilateral L5, S1, S2, and S3 radi..."
4,4,"OPERATIVE PROCEDURES: , Colonoscopy and biopsi..."


In [ ]:
training_data = pd.read_csv('/content/drive/MyDrive/Intact data/new_train (1).csv')

In [ ]:
training_data.head()

,Unnamed: 0,medical_specialty,transcription,labels
0,0,Emergency Room Reports,"REASON FOR THE VISIT:, Very high PT/INR.,HIST...",0
1,1,Surgery,"PREOPERATIVE DIAGNOSIS:, Acetabular fracture ...",1
2,2,Surgery,"NAME OF PROCEDURE,1. Selective coronary angio...",1
3,3,Radiology,"REFERRING DIAGNOSIS: , Motor neuron disease.,P...",2
4,4,Emergency Room Reports,"CHIEF COMPLAINT: , Dental pain.,HISTORY OF PRE...",0


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 20.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-sci-md: filename=en_core_sci_md-0.5.1-py3-none-any.whl size=120253165 sha256=dda5059eb024b93bf77265e9219f6fd1b9724202d8835800d6a0f3da09f817c1
  Stored in directory: /root/.cache/pip/wheels/cd/2d/59/43791dba56f306a2eaa9b510a42699a61e992344837bd25b3f
Successfully built en-core-sci-md


In [ ]:
import scispacy
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp = spacy.load('en_core_sci_md')

In [ ]:
doc = nlp(training_data['transcription'][0])
print(doc.ents)

(REASON, patient, lady, month, pneumonia, CHF, atrial fibrillation, chronic problem, Coumadin, started, INR, regulate, Coumadin, Coumadin, outpatient, INR, emergency room, vitamin K, anticoagulation.,PAST, HISTORY:,1, Congestive heart, failure.,2, Coronary artery disease.,4, pneumonia.,7, ruptured, colon.,9, Digoxin.,6, Triamterene, hydrochlorothiazide.,9, smoke, distress.,VITAL, pulse, respirations, temperature, auscultation, percussion.,HEART, Coagulopathy, induced, Coumadin.,PLAN, INR, office, elevated, vitamin K, D5W, home, PT/INR, next week, Coumadin, case, age, comorbidities, multiple, medications, adequate, level, anticoagulation, fall, aspirin, day, anticoagulation, risk, stroke, anticoagulation, Coumadin, Coumadin, time, I, outpatient)


In [ ]:
print(training_data.shape[0]/3)

1323.0


In [ ]:
training_data.shape

(3969, 4)

In [ ]:
testing_data.shape

(997, 2)

In [ ]:
all_transcriptions = pd.concat([testing_data['transcription'],training_data['transcription']], ignore_index=True)

In [ ]:
all_transcriptions[500]

'PROCEDURE: , Bilateral L5 dorsal ramus block and bilateral S1, S2, and S3 lateral branch block.,INDICATION: , Sacroiliac joint pain.,INFORMED CONSENT: , The risks, benefits and alternatives of the procedure were discussed with the patient.  The patient was given opportunity to ask questions regarding the procedure, its indications and the associated risks.,The risk of the procedure discussed include infection, bleeding, allergic reaction, dural puncture, headache, nerve injuries, spinal cord injury, and cardiovascular and CNS side effects with possible vascular entry of medications.  I also informed the patient of potential side effects or reactions to the medications potentially used during the procedure including sedatives, narcotics, nonionic contrast agents, anesthetics, and corticosteroids.,The patient was informed both verbally and in writing.  The patient understood the informed consent and desired to have the procedure performed.,PROCEDURE:  ,Oxygen saturation and vital signs 

In [ ]:
#feature engineering on training data
total_texts = []


for x in range(all_transcriptions.shape[0]):
  dat = nlp(all_transcriptions[x])
  data = dat.ents
  # print(data)
  # print(type(data))

  texts = []

  for x in data:
    text = x.text
    texts.append(text)
 
  listToStr = ' '.join([elem for i,elem in enumerate(texts)])

  total_texts.append(listToStr)




  


In [ ]:
print(len(total_texts))

4966


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
count_mat = vectorizer.fit_transform(total_texts)

In [ ]:
count_mat.shape

(4966, 18780)

In [ ]:
training_data.groupby('medical_specialty').describe()

Unnamed: 0                                    \
                                    count         mean          std     min   
medical_specialty                                                             
 Allergy / Immunology                 6.0  2162.666667  1405.818433   406.0   
 Autopsy                              7.0  1860.142857  1413.468598    73.0   
 Bariatrics                          15.0  2055.333333  1030.413209   203.0   
 Cardiovascular / Pulmonary         309.0  2059.055016  1197.402232    13.0   
 Chiropractic                        12.0  1583.666667  1178.446846    28.0   
 Consult - History and Phy.         410.0  1918.312195  1143.851979    41.0   
 Cosmetic / Plastic Surgery          19.0  1842.368421  1094.790706    25.0   
 Dentistry                           21.0  1940.142857  1062.123029   162.0   
 Dermatology                         21.0  2043.095238  1255.620600    79.0   
 Diets and Nutritions                 9.0  2778.444444  1030.799339   714.0   
 Discharge Summary                   87.0  1771.195402  1129.832523    71.0   
 ENT - Otolaryngology                82.0  2084.597561  1148.877733    19.0   
 Emergency Room Reports              63.0  1779.285714  1264.238090     0.0   
 Endocrinology                       16.0  1952.000000   982.273282   220.0   
 Gastroenterology                   176.0  2062.835227  1202.559965     8.0   
 General Medicine                   209.0  2042.684211  1147.976441    20.0   
 Hematology - Oncology               68.0  2242.897059  1255.735643    22.0   
 Hospice - Palliative Care            6.0  1897.166667  1483.112324    47.0   
 IME-QME-Work Comp etc.              12.0  2040.166667  1278.907761   398.0   
 Lab Medicine - Pathology             5.0  1474.000000  1341.887663   147.0   
 Letters                             19.0  1815.526316  1403.444982   121.0   
 Nephrology                          63.0  2158.873016  1135.995094    14.0   
 Neurology                          170.0  2172.382353  1217.813955     7.0   
 Neurosurgery                        71.0  1961.281690  1154.994387    48.0   
 Obstetrics / Gynecology            123.0  1885.813008  1108.168770    50.0   
 Office Notes                        38.0  2086.552632  1051.739632   138.0   
 Ophthalmology                       67.0  1940.089552  1200.304327   148.0   
 Orthopedic                         289.0  2033.685121  1097.717301    12.0   
 Pain Management                     54.0  2001.814815  1042.204106   336.0   
 Pediatrics - Neonatal               55.0  2125.927273  1228.086133   171.0   
 Physical Medicine - Rehab           16.0  2174.687500  1229.361770   190.0   
 Podiatry                            35.0  1749.057143  1146.423876     5.0   
 Psychiatry / Psychology             45.0  1812.200000  1119.614495    29.0   
 Radiology                          213.0  2105.779343  1103.820216     3.0   
 Rheumatology                         6.0  2553.833333   876.024981  1285.0   
 SOAP / Chart / Progress Notes      135.0  2030.140741  1086.258443    27.0   
 Sleep Medicine                      12.0  1428.333333   827.893201   504.0   
 Speech - Language                    8.0  1785.250000  1318.661090   160.0   
 Surgery                            863.0  1946.858633  1154.784355     1.0   
 Urology                            134.0  2037.776119  1186.166686    52.0   

                                                                 labels        \
                                    25%     50%      75%     max  count  mean   
medical_specialty                                                               
 Allergy / Immunology           1052.50  2262.5  3294.00  3758.0    6.0  36.0   
 Autopsy                         886.00  1653.0  2827.50  3868.0    7.0  22.0   
 Bariatrics                     1283.00  2107.0  2775.50  3455.0   15.0  32.0   
 Cardiovascular / Pulmonary      955.00  2187.0  3125.00  3993.0  309.0   7.0   
 Chiropractic                    699.50  1483.0  2332.25  3897.0  

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf = TfidfTransformer().fit(count_mat)

In [ ]:
mat_tfidf = tfidf.transform(count_mat)

In [ ]:
mat_tfidf

<4966x18780 sparse matrix of type '<class 'numpy.float64'>'
	with 580696 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
training_data['labels']

0       0
1       1
2       1
3       2
4       0
       ..
3964    4
3965    1
3966    1
3967    5
3968    1
Name: labels, Length: 3969, dtype: int64

In [ ]:


classifier = MultinomialNB().fit(mat_tfidf[0:3969,:],training_data['labels'])

In [ ]:
mat_tfidf[3970:,].shape

(996, 18780)

In [ ]:
predictions = classifier.predict(mat_tfidf[3970:,])